In [13]:
import threading


def thread_hello():
    other = threading.Thread(target=thread_say_hello, args=())
    other.start()
    thread_say_hello()


def thread_say_hello():
    print('hello from', threading.current_thread().name)


thread_hello()

hello fromhello from MainThread
 Thread-10 (thread_say_hello)


In [12]:
import multiprocessing


def process_hello():
    other = multiprocessing.Process(target=process_say_hello, args=())
    other.start()
    process_say_hello()


def process_say_hello():
    print('hello from', multiprocessing.current_process().name)


process_hello()

hello from MainProcess


In [14]:
items = []
flag = []


def consume():
    while not flag:
        pass
    print('items is', items)


def produce():
    consumer = threading.Thread(target=consume, args=())
    consumer.start()
    for i in range(10):
        items.append(i)
    flag.append('go')


produce()

items is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
from queue import Queue

queue = Queue()


def synchronized_consume():
    while True:
        print('got an item:', queue.get())
        queue.task_done()


def synchronized_produce():
    consumer = threading.Thread(target=synchronized_consume(), args=())
    consumer.daemon = True
    consumer.start()
    for i in range(10):
        queue.put(i)
    queue.join()


synchronized_produce()